#### Tratamento de dados para camada silver

#### Tabela dm artilheiro

In [7]:

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

# Iniciando uma SparkSession
spark = SparkSession.builder.appName("etl_silver").getOrCreate()

StatementMeta(, 6dced942-db4a-4e95-a898-f0bdf962a7eb, 9, Finished, Available, Finished)

In [18]:
df_artilheiro = spark.read.load(
    'Files/Bronze/dm_artilheiros',
    format='parquet',
    header=True
    )

StatementMeta(, 6dced942-db4a-4e95-a898-f0bdf962a7eb, 20, Finished, Available, Finished)

In [ ]:
# renomeando os nomes das colunas
df_artilheiro = df_artilheiro.selectExpr(
    "Id_jogador as Id_jogador",
    "Position as Posicao",
    "Name as Nome",
    "Team as Id_time",
    "PlayedMatches as Total_partidas",
    "Goals as Gols",
    "Assists as Assistencias",
    "Penalties as Penaltis")


In [22]:
# salvando tabela de dm_artilheiro na camada silver e criando uma tabela delta

df_artilheiro.write.mode("overwrite").parquet('Files/Silver/dm_artilheiros')
df_artilheiro.write.format("delta").mode("overwrite").saveAsTable("dm_artilheiros")

StatementMeta(, 6dced942-db4a-4e95-a898-f0bdf962a7eb, 24, Finished, Available, Finished)

#### Tabela dm partidas

In [5]:
# Criando data frame da dm partidas
df_partidas = spark.read.load(
    'Files/Bronze/dm_partidas',
    format='parquet',
    header=True
    )

StatementMeta(, 434b705a-f0a0-4cc5-ba57-8a661291c831, 7, Finished, Available, Finished)

In [6]:
df_partidas = df_partidas.withColumnRenamed("Time Casa", "Time_Casa")
df_partidas = df_partidas.withColumnRenamed("Time Visitante", "Time_Visitante")

StatementMeta(, 434b705a-f0a0-4cc5-ba57-8a661291c831, 8, Finished, Available, Finished)

In [7]:
# alterando valores da coluna status 

df_partidas = df_partidas.withColumn(
    "Status", 
    when(df_partidas["Status"] == "FINISHED", "FINAL")
    .when(df_partidas["Status"] == "POSTPONED", "POSTERGADA")
    .when(df_partidas["Status"] == "TIMED", "TEMPORIZADA")
    .when(df_partidas["Status"] == "SCHEDULED", "AGENDADA")
    .when(df_partidas["Status"] == "PAUSED", "PAUSADA")   
    .otherwise(df_partidas["Status"])
    )

StatementMeta(, 434b705a-f0a0-4cc5-ba57-8a661291c831, 9, Finished, Available, Finished)

In [9]:
# salvando tabela de dm_partidas na camada silver e criando uma tabela delta

df_partidas.write.mode("overwrite").parquet('Files/Silver/dm_partidas')
df_partidas.write.format("delta").mode("overwrite").saveAsTable("dm_partidas")

StatementMeta(, 434b705a-f0a0-4cc5-ba57-8a661291c831, 11, Finished, Available, Finished)

#### Tabela dm clubes

In [10]:
# Criando data frame da dm clubes
df_clubes = spark.read.load(
    'Files/Bronze/dm_clubes',
    format='parquet',
    header=True
    )

StatementMeta(, 434b705a-f0a0-4cc5-ba57-8a661291c831, 12, Finished, Available, Finished)

In [11]:
#  criando uma tabela delta
df_clubes.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("dm_clubes")

StatementMeta(, 434b705a-f0a0-4cc5-ba57-8a661291c831, 13, Finished, Available, Finished)

#### Tabela dm tabela

In [12]:
# Criando data frame da dm tabela
df_tabela = spark.read.load(
    'Files/Bronze/dm_tabela',
    format='parquet',
    header=True
    )

StatementMeta(, 434b705a-f0a0-4cc5-ba57-8a661291c831, 14, Finished, Available, Finished)

In [13]:
#  criando uma tabela delta
df_tabela.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("dm_tabela")

StatementMeta(, 434b705a-f0a0-4cc5-ba57-8a661291c831, 15, Finished, Available, Finished)

#### Tabela jogadores

In [2]:
df_jogadores = spark.read.load(
    'Files/Bronze/dm_jogadores',
    format = 'parquet',
    header=True
)

StatementMeta(, 4bcb8e37-d99a-4c27-bf14-7afccdaee442, 4, Finished, Available, Finished)

In [14]:
# Traduzindo as nacionalidade dos jogadores

df_jogadores = df_jogadores.withColumn(
    "nacionalidade", 
    when(df_jogadores["nacionalidade"] == "Ecuador", "Equador")
    .when(df_jogadores["nacionalidade"] == "Uruguay", "Uruguai")
    .when(df_jogadores["nacionalidade"] == "Brazil", "Brasil")
    .when(df_jogadores["nacionalidade"] == "Spain", "Espanha")
    .when(df_jogadores["nacionalidade"] == "Italy", "Italia")   
    .when(df_jogadores["nacionalidade"] == "France", "França")   
    .otherwise(df_jogadores["nacionalidade"])
    )


StatementMeta(, 4bcb8e37-d99a-4c27-bf14-7afccdaee442, 16, Finished, Available, Finished)

In [15]:
# Traduzindo nome as posições dos jogadores
df_jogadores = df_jogadores.withColumn(
    "posicao",
    when(df_jogadores["posicao"] == "Centre-Back", "Zagueiro")
    .when(df_jogadores["posicao"] == "Defence", "Zagueiro")
    .when(df_jogadores["posicao"] == "Right-Back", "Lateral Direito")
    .when(df_jogadores["posicao"] == "Central Midfield", "Meio Campo")
    .when(df_jogadores["posicao"] == "Midfield", "Meio Campo")   
    .when(df_jogadores["posicao"] == "Offence", "Atacante") 
    .when(df_jogadores["posicao"] == "Right Winger", "Atacante") 
    .when(df_jogadores["posicao"] == "Goalkeeper", "Goleiro") 
    .when(df_jogadores["posicao"] == "Left Winger", "Atacante") 
    .when(df_jogadores["posicao"] == "Midfielder", "Meio Campo") 
    .when(df_jogadores["posicao"] == "Keeper", "Goleiro") 
    .when(df_jogadores["posicao"] == "Attacking Midfield", "Meio Campo Ofensivo")
    .when(df_jogadores["posicao"] == "Defensive Midfield", "Volante") 
    .when(df_jogadores["posicao"] == "Forward", "França")  
    .when(df_jogadores["posicao"] == "Left-Back", "Lateral Esquerdo")  
    .when(df_jogadores["posicao"] == "Defender", "Zagueiro")  
    .when(df_jogadores["posicao"] == "Centre-Forward", "Atacante") 
    .otherwise(df_jogadores["posicao"])
    )

StatementMeta(, 4bcb8e37-d99a-4c27-bf14-7afccdaee442, 17, Finished, Available, Finished)

In [18]:
# Salvando em tabela dm_jogadores

df_jogadores.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("dm_jogadores")

StatementMeta(, 4bcb8e37-d99a-4c27-bf14-7afccdaee442, 20, Finished, Available, Finished)

#### 